# 如何使用Chat Completions API 参数 functions

## 使用 openai

In [1]:
import openai

GPT_MODEL = "gpt-3.5-turbo-0613"

function_descriptions = [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]

user_query = "What is the weather like in Boston?"

### 把问题与函数定义交给 function call，自动选择合适的函数

In [3]:
response = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[{"role": "user", "content": user_query}],
    functions=function_descriptions,
    function_call="auto",
)
response_message = response["choices"][0]["message"]
print(response_message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
  }
}


In [ ]:
### 调用 function call 选择的函数，得到返回值

In [7]:
import json
import inspect

def get_function_parameter_names(function):
  if function is not None and inspect.isfunction(function):
      parameter_names = inspect.signature(function).parameters.keys()
      return list(parameter_names)
  else:
      return None

# Locate the function and make the call
def cal_function(_function_name, _arguments):
    the_function = globals().get(_function_name)
    parameter_names = get_function_parameter_names(the_function)
    parameter_values = []
    for parameter_name in parameter_names:
      parameter_values.append(_arguments[parameter_name])
    return the_function(*parameter_values)

def get_current_weather(location):
    return json.dumps({ "temperature": 22, "unit": "celsius", "description": "Sunny", "location": location })

function_name = response_message["function_call"]["name"]
arguments = json.loads(response_message["function_call"]["arguments"])
returned_value = cal_function(function_name, arguments)
print(returned_value)

{"temperature": 22, "unit": "celsius", "description": "Sunny", "location": "Boston, MA"}


In [ ]:
### 将响应再次发送给模型，让其进行总结

In [8]:
second_response = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[
        {"role": "user", "content": user_query},
        response_message,
        {
            "role": "function",
            "name": "get_current_weather",
            "content": returned_value,
        },
    ],
)
print(second_response)

{
  "id": "chatcmpl-7ler8OVrOnxd76Ovb8xkyyQxo9x2K",
  "object": "chat.completion",
  "created": 1691592822,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Boston is currently sunny with a temperature of 22 degrees Celsius."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 70,
    "completion_tokens": 16,
    "total_tokens": 86
  }
}


## 使用 LangChain Agent


In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool

# 在 function description 中描述参数结构，LLM 可能会自动识别，但最好还是配置 args_schema
@tool("get_current_weather")
def get_current_weather(location):
    """Get the current weather in a given location"""
    return json.dumps({ "temperature": 22, "unit": "celsius", "description": "Sunny", "location": location })

# 工具是给 agent 自动分析使用
tools = [get_current_weather]

question_format_str = "What is the weather like in Boston?"

from langchain.agents import initialize_agent
from langchain.agents import AgentType

llm = ChatOpenAI(model_name=GPT_MODEL, temperature=0)

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_MULTI_FUNCTIONS, verbose=True)
agent_response = agent.run(question_format_str)
print(agent_response)



> Entering new AgentExecutor chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).



Invoking: `get_current_weather` with `{'location': 'Boston'}`


{"temperature": 22, "unit": "celsius", "description": "Sunny", "location": "Boston"}The current weather in Boston is sunny with a temperature of 22 degrees Celsius.

> Finished chain.
The current weather in Boston is sunny with a temperature of 22 degrees Celsius.
